In [59]:
{-# LANGUAGE TemplateHaskell #-}
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE TypeFamilies #-}

import           Lens.Micro.TH (makeLenses)
import           Lens.Micro (Lens')--, set)
import           Lens.Micro.Extras (view)

import           Data.Text (Text, lines, unpack)--, pack)
import qualified Data.Text.IO as Text (readFile)
import           Data.Maybe (mapMaybe)
import           Text.Megaparsec
import qualified Text.Megaparsec.Lexer as L

In [83]:
data Song = Song
  {
    _genre          :: String,
    _trackId        :: String,
    _artistName     :: String,
    _title          :: String,

    _loudness       :: Double,
    _tempo          :: Double,
    _timeSignature  :: Integer,
    _key            :: Integer,
    _mode           :: Integer,
    _duration       :: Double,
    _avgTimbre      :: [Double], -- there are only 12 avg_timbre numbers
    _varTimbre      :: [Double]  -- there are only 12 var_timbre numbers
  } deriving (Eq, Show)

makeLenses ''Song

In [120]:
line2song :: Text -> Maybe Song
line2song = parseMaybe songParser

songParser :: Parsec Dec Text Song
songParser = Song <$> (cell <* comma)
                  <*> (cell <* comma)
                  <*> (cell <* comma)
                  <*> (cell <* comma)
                  <*> (double <* comma)
                  <*> (double <* comma)
                  <*> (integer <* comma)
                  <*> (integer <* comma)
                  <*> (integer <* comma)
                  <*> (double <* comma)
                  <*> (count 12 (double <* comma))
                  <*> (sepBy double comma) -- there should be 12 in here

  where
    --cell :: Parsec Dec Text [Char]
    cell = some (noneOf [','])
    --comma :: Parsec Dec Text ()
    comma = char ',' >> return ()
    --double :: Parsec Dec Text Double
    double  = L.signed (return ()) L.float
    integer = L.signed (return ()) L.integer

In [5]:
filename = "./msd_genre_dataset.txt"

In [122]:
file <- Text.readFile filename
let fileLines = Data.Text.lines file
let songs = mapMaybe line2song $ drop 10 fileLines

In [127]:
mapM_ print . take 3 $ drop 60000 songs